# Channel evolution analysis

In [1]:
import os
import sys
import pandas as pd
import pyarrow.feather as feather
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
tqdm.pandas()

In [2]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from utils.ploting import *

In [3]:
NOTEBOOK_PATH = os.getcwd()
MAIN_PATH = os.path.dirname(NOTEBOOK_PATH)
DATA_PATH = os.path.join(MAIN_PATH, 'data_youniverse')
SITE_DATA_PATH = os.path.join(MAIN_PATH, 'website_data')
PEOPLE_AND_BLOGS_PATH = os.path.join(SITE_DATA_PATH, 'People_&_Blogs')

if not os.path.exists(PEOPLE_AND_BLOGS_PATH):
    os.makedirs(PEOPLE_AND_BLOGS_PATH)

In [7]:
pd.read_csv(os.path.join(PEOPLE_AND_BLOGS_PATH, 'df_PB_channels.csv'), compression='infer', index_col=0)


,category_cc,join_date,channel,name_cc,subscribers_cc,videos_cc,subscriber_rank_sb,weights
89,People & Blogs,2013-09-19,UCcgVECVN4OKV6DH1jLkqmcA,Jake Paul,19600000,824,144.0,2.0870
98,People & Blogs,2011-08-10,UCpko_-a4wgz2u_DgDgd9fqA,BuzzFeedVideo,19400000,6334,158.0,2.0870
120,People & Blogs,2016-01-10,UCWwWOFsW68TqXE-HZLC3WIA,The ACE Family,17600000,460,185.0,2.0870
137,People & Blogs,2006-12-06,UCAuUUnT6oDeKwE6v1NGQxug,TED,14800000,3105,211.0,2.0870
154,People & Blogs,2008-05-15,UCNL1ZadSjHpjm4q9j2sVtOA,Lady Gaga,14495273,163,238.0,2.0870
...,...,...,...,...,...,...,...,...
136442,People & Blogs,2018-07-11,UC9mdbZJBLOWQkHIpoaJPjeQ,_Nevaeh Crockett_,10100,54,996760.0,53.1435
136461,People & Blogs,2016-12-27,UCUe7JH_tgicQsaL2grkqrrQ,Multi-Fandom Girl,10000,12,1006793.0,53.1435
136462,People & Blogs,2014-06-25,UC8jUn5SLi5dwjE5v6wY0UOg,Anastasiya Balan,10000,12,1007083.0,53.1435
136466,People & Blogs,2013-10-17,UCtW9jp5TH0YrgYpwiRf9t-Q,saidthestory,10100,352,1008644.0,53.1435


In [27]:
pd.read_csv(os.path.join(PEOPLE_AND_BLOGS_PATH, 'df_PB_helper.csv.gz'), compression='gzip')

,level_0,index,categories,channel_id,dislike_count,display_id,duration,like_count,upload_date,view_count,year_month,like_rate
0,0,1783,People & Blogs,UCzWmWTPCafQRNl9q9Y53AfQ,613.0,BDnF-EgqAGw,398,49464.0,2019-02-05,1142835.0,2019-02,0.987759
1,1,1784,People & Blogs,UCzWmWTPCafQRNl9q9Y53AfQ,588.0,wrTIXbQED80,375,32726.0,2019-01-31,819654.0,2019-01,0.982350
2,2,1785,People & Blogs,UCzWmWTPCafQRNl9q9Y53AfQ,1177.0,uyWL0ZWibyU,335,86222.0,2019-01-22,2159837.0,2019-01,0.986533
3,3,1786,People & Blogs,UCzWmWTPCafQRNl9q9Y53AfQ,673.0,yd7d51AgSso,319,30596.0,2019-01-15,736707.0,2019-01,0.978477
4,4,1787,People & Blogs,UCzWmWTPCafQRNl9q9Y53AfQ,991.0,-Ia18bqO-9w,367,75504.0,2019-01-09,1636316.0,2019-01,0.987045
...,...,...,...,...,...,...,...,...,...,...,...,...
5553832,5553832,72904887,People & Blogs,UCrwRLYg4dbkrxDbbnaQd48g,1.0,Z3mXSYy3qBk,165,19.0,2016-03-31,426.0,2016-03,0.950000
5553833,5553833,72904888,People & Blogs,UCrwRLYg4dbkrxDbbnaQd48g,0.0,fiHAotJWQ1k,148,29.0,2016-03-31,659.0,2016-03,1.000000
5553834,5553834,72904889,People & Blogs,UCrwRLYg4dbkrxDbbnaQd48g,1.0,-sVZEtcIB0I,188,32.0,2016-03-31,863.0,2016-03,0.969697
5553835,5553835,72904890,People & Blogs,UCrwRLYg4dbkrxDbbnaQd48g,1.0,Wi4dcBmmzK8,144,137.0,2016-03-18,2528.0,2016-03,0.992754


In [ ]:
df_feather = feather.read_feather(os.path.join(DATA_PATH, 'yt_metadata_helper.feather')) # 9m : 

In [8]:
channels = pd.read_csv(os.path.join(DATA_PATH, "df_channels_en.tsv.gz"), compression="infer", sep="\t")

In [ ]:
timeseries = pd.read_csv(f"{DATA_PATH}/df_timeseries_en.tsv.gz", compression="infer", sep="\t") # 20s
timeseries.head(1)

In [10]:
channels["join_date"] = pd.to_datetime(channels["join_date"])
timeseries["datetime"] = pd.to_datetime(timeseries["datetime"])
df_feather["upload_date"] = pd.to_datetime(df_feather["upload_date"])
df_feather["year_month"] = df_feather['upload_date'].dt.to_period('M')
df_feather["like_rate"] = df_feather["like_count"] / (df_feather["like_count"] + df_feather["dislike_count"])
channels_PB = channels[channels['category_cc'] == "People & Blogs"]
df_feather_PB = df_feather[df_feather["channel_id"].isin(channels_PB["channel"])]
timeseries_PB = timeseries[timeseries['channel'].isin(channels_PB['channel'])]
df_feather.head(1)

In [ ]:
# import string

# # Assuming df_channels is your dataframe and 'name_cc' is the column with channel names
# # Create a temporary column 'name_cc_sorted' that has punctuation removed
# channels_PB['name_cc_sorted'] = channels_PB['name_cc'].str.translate(str.maketrans('', '', string.punctuation))

# # Sort the dataframe using the new column
# channels_PB.sort_values(by='name_cc_sorted', inplace=True)

# # Drop the temporary sorting column
# channels_PB.drop(columns=['name_cc_sorted'], inplace=True)

# channels_PB.head()

In [24]:
df_feather_PB.reset_index().to_csv(os.path.join(PEOPLE_AND_BLOGS_PATH, 'df_PB_helper.csv.gz'), compression='gzip', index=False)
channels_PB.to_csv(os.path.join(PEOPLE_AND_BLOGS_PATH,"df_PB_channels.csv.gz"), compression="gzip", index=False)

In [ ]:
channels_PB.sample(1)

In [ ]:
def filter_not_diverse_channels(channel_id, df_feather=df_feather_PB, show_evolution=False):
    '''
    Calculates the proportion of videos in the 'People & Blogs' category for a given channel.

    Parameters:
    - channel_id (str): The YouTube channel ID.
    - df_feather (DataFrame): DataFrame containing video data.

    Returns:
    - bool: True if the proportion of 'People & Blogs' videos is less than 75%, False otherwise.
    '''
    df_filtered = df_feather[df_feather["channel_id"] == channel_id]
    grouped_data = df_filtered.groupby(["year_month", "categories"]).size()
    # display(grouped_data)
    sorted_data = grouped_data.reset_index(name='count').sort_values(['year_month', 'count'], ascending=[True, False])
    if show_evolution:
        display(sorted_data)

    most_popular_cat = sorted_data.drop_duplicates(subset=["year_month"])
    # display(most_popular_cat)

    people_and_blog_proportion = (most_popular_cat["categories"] == "People & Blogs").mean()
    # print(people_and_blog_proportion)
    return people_and_blog_proportion < 0.75

In [ ]:
i = 100
channels_PB_transi_mask = channels_PB[:i].progress_apply(lambda channel: filter_not_diverse_channels(channel["channel"]), axis=1)
channels_PB_transi = channels_PB[:i][channels_PB_transi_mask]
print(f"There are {len(channels_PB[:i])} channels in PB category, but only {len(channels_PB_transi)} have less than 0.75 video in that category")
# channels_PB_transi.sample(30)

In [ ]:
for channel_id in channels_PB_transi['channel'].unique()[:5]:
    visualize_evolution_of_channel(channel_id, df_feather_PB, channels_PB)


In [ ]:
channel_ids = ['UC4-CH0epzZpD_ARhxCx6LaQ', 'UCYd0us2OtW4d4-1cfpT2ktw']
# for channel_id in channel_ids:
#     visualize_evolution_of_channel(channel_id, df_feather_PB, channels_PB)

visualize_evolution_of_channel(channel_ids[0], df_feather_PB, channels_PB, start_date="2016-09", end_date="2019-09", transition_date="2018-02")

In [ ]:
channel_ids = ['UC4-CH0epzZpD_ARhxCx6LaQ', 'UC0oRdyzTbGIZUISiGQnDdiw']
video_frequency_and_duration(channel_ids, df_feather, channels, start_date="2016-09", end_date="2019-09", transition_date="2018-03")

In [ ]:
channel_ids = ['UC4-CH0epzZpD_ARhxCx6LaQ', 'UC0oRdyzTbGIZUISiGQnDdiw']
video_likes_and_views(channel_ids, df_feather, channels, start_date="2016-09", end_date="2019-09", transition_date="2018-03")

# 2nd milestone stuff

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np
# import os 
# import pandas as pd
# import seaborn as sns
# import sys

In [ ]:
# def category_evolution_of_channel_v0(channel_id, channels=channels_PB, df_feather=df_feather_PB):
#     '''
#     Input : a channel id, channels information dataframe and video metadata via df_feather
#     Output : annual count of number of videos of that channel in every category
#     Note : df_feather['upload_date'] should be already in datetime format
#     '''
#     # get info on the channel
#     # channel_info = channels[channels["channel"] == channel_id]
#     # display(channel_info)

#     df_feather_filt = df_feather[df_feather["channel_id"] == channel_id]
#     # For every year, count videos in each category
#     grouped_data = df_feather_filt[["year_month", "categories", "channel_id"]]
#     grouped_data = grouped_data.groupby(["year_month", "categories"]).count()
#     # Sort the values (need to transform in a regular dataframe and then put back indexes)
#     sorted_data = grouped_data.reset_index().sort_values(['year_month', 'channel_id'], ascending=[True, False])
#     sorted_data.set_index(['year_month', 'categories'], inplace=True)
#     # display(sorted_data)

#     # Select only the most popular category every year
#     most_popular_cat = sorted_data.reset_index().drop_duplicates(subset=["year_month"])
#     # display(most_popular_cat)

#     people_and_blog_proportion = len(most_popular_cat[most_popular_cat["categories"]=="People & Blogs"]) / len(most_popular_cat)
#     # print(people_and_blog_proportion)

#     return people_and_blog_proportion < 0.75

# # channel_id = 'UCcgVECVN4OKV6DH1jLkqmcA'
# # category_evolution_of_channel(channel_id)

In [ ]:
# NOTEBOOK_PATH = os.getcwd()
# DIR_PATH = os.path.dirname(NOTEBOOK_PATH)
# DATA_PATH = os.path.join(DIR_PATH, "data_youniverse")
# UTILS_PATH = os.path.join(DIR_PATH, "utils")
# CATEGORY = "Science & Technology"

# # append utils path and import utils
# sys.path.append(UTILS_PATH)
# from ploting import *
# from loading import *

In [ ]:
# df_channels_en = pd.read_csv(f"{DATA_PATH}/df_channels_en.tsv.gz", compression="infer", sep="\t") 
# # df_channels_en.head()

In [ ]:
# df_timeseries_en = pd.read_csv(f"{DATA_PATH}/df_timeseries_en.tsv.gz", compression="infer", sep="\t") # 20s
# # df_timeseries_en.head()

In [ ]:
# df_timeseries_en[df_timeseries_en['category'].isna()].channel.unique().size

In [ ]:
# df_channels_filt = df_channels_en[df_channels_en['category_cc'] == CATEGORY].copy()
# df_timeseries_filt = df_timeseries_en[df_timeseries_en['category'] == CATEGORY].copy()

In [ ]:
# helper_filt = read_and_filter_feather(f"{DATA_PATH}/yt_metadata_helper.feather", CATEGORY)
# helper_filt.head()

In [ ]:
# # UNCOMMENT IF NEEDED
# yt_metadata_filtered = load_and_filter_jsonl_gz_file_by_chunks(f"{DATA_PATH}/yt_metadata_en.jsonl.gz", CATEGORY) # 21m for pets and animals (should be the same for the other at it is reading all the file anyway)
# yt_metadata_filtered.to_csv(f"{DATA_PATH}/yt_metadata_pets&animals.csv.gz", index=False, compression="gzip") # stock the results
# yt_metadata_filtered.head()

In [ ]:
# yt_metadata_filtered.iloc[-1]['tags']

### Analysis of pets & animals category and tags

In [ ]:
# yt_metadata_filt = pd.read_csv(f"{DATA_PATH}/yt_metadata_pets&animals.csv.gz", compression="infer")
# # les channels id sont les vrai, ont peut retrouver les chaines youtube !

In [ ]:
# yt_metadata_filt['upload_date'] = pd.to_datetime(yt_metadata_filt['upload_date'])

In [ ]:
# yt_metadata_filt.describe()

### Plot the top 3 tags for a range of years

In [ ]:
# years = range(2016, 2020)  # or whatever years are applicable
# for year in years:
#     yt_metadata_filt_year = yt_metadata_filt[yt_metadata_filt['upload_date'].dt.year == year]
#     if not yt_metadata_filt_year.empty:
#         try:
#             # Assuming the function plot_topN_tag is correctly defined and can handle cases with less than N tags
#             plot_topN_tag(yt_metadata_filt_year['tags'], 3)
#         except Exception as e:
#             print(f"An error occurred for year {year}: {e}")
#     else:
#         print(f"No data available for year {year}")

### Cleaning of NaN values (not extensive)

In [ ]:
# def clean_dataframe(df):
#     # Remove rows where 'tags' is empty or NaN
#     initial_row_count = len(df)
#     df = df.dropna(subset=['tags'])  # Drops rows with NaN in 'tags'
#     df = df[df['tags'].str.strip().astype(bool)]  # Drops rows with empty 'tags'
#     cleaned_row_count = len(df)

#     # Check for NaN values in 'like_count'
#     like_count_nan = df['like_count'].isna().sum()
#     if like_count_nan > 0:
#         # Handle NaN values here. Options: fill with 0, mean, median, etc.
#         # Example: df['like_count'].fillna(df['like_count'].median(), inplace=True)
#         # For now, we'll just drop these rows
#         df = df.dropna(subset=['like_count'])

#     # Print out the cleaning summary
#     print(f"Rows with empty 'tags' removed: {initial_row_count - cleaned_row_count}")
#     print(f"Rows with NaN in 'like_count': {like_count_nan}")

#     return df

# # Clean the yt_metadata_filtered dataframe
# yt_metadata_filt_clean = clean_dataframe(yt_metadata_filt)

### Plot evolution of most popular tags

In [ ]:
# def get_top_tags_for_year(df, year, top_n=5):
#     """ Get the top N tags for a given year from the dataframe """
#     year_data = df[df['upload_date'].dt.year == year]
#     tag_list = year_data['tags'].str.split(',').explode()
#     top_tags = tag_list.value_counts().head(top_n).index
#     return top_tags

# def count_tag_appearances_by_year(df, tags):
#     """ Count the number of appearances of each tag by year """
#     df['year'] = df['upload_date'].dt.year
#     tag_counts_by_year = {tag: df[df['tags'].str.contains(tag)]['year'].value_counts().sort_index() for tag in tags}
#     return tag_counts_by_year

# def plot_tag_counts(tag_counts_by_year):
#     """ Plot the evolution of tag counts over the years """
#     plt.figure(figsize=(10, 8))
#     for tag, counts in tag_counts_by_year.items():
#         plt.plot(counts.index, counts.values, label=tag)
    
#     plt.xlabel('Year')
#     plt.ylabel('Number of Appearances')
#     plt.title('Evolution of Tag Appearances by Year')
#     plt.legend()
#     plt.show()

# top_tags_2019 = get_top_tags_for_year(yt_metadata_filt_clean, 2019)
# tag_counts_by_year = count_tag_appearances_by_year(yt_metadata_filt_clean, top_tags_2019)
# plot_tag_counts(tag_counts_by_year)

### Dataframe of statistics grouped by year

In [ ]:
# # Convert `upload_date` to DateTime and extract the year.
# yt_metadata_filt['upload_year'] = pd.to_datetime(yt_metadata_filt['upload_date']).dt.year

# # Group by the extracted year.
# grouped = yt_metadata_filt.groupby('upload_year')

# # Initialize a dictionary to hold aggregated data.
# aggregated_data = {}

# # Iterate over each group to perform aggregations.
# for name, group in grouped:
#     # Get the most popular tag for each year.
#     popular_tag = group['tags'].str.split(',').explode().value_counts().idxmax()
    
#     # Get other statistics for each year.
#     aggregated_data[name] = {
#         'most_popular_tag': popular_tag,
#         'total_like_count': group['like_count'].sum(),
#         'total_videos': group['upload_date'].count(),
#         'total_duration': group['duration'].sum(),
#         'unique_channels': group['channel_id'].nunique(),
#         'total_dislike_count': group['dislike_count'].sum(),
#         'total_view_count': group['view_count'].sum(),
#         'mean_dislike_count': group['dislike_count'].mean(),
#         'mean_like_count': group['like_count'].mean(),
#         'mean_duration': group['duration'].mean(),
#     }

# # Convert aggregated data to a DataFrame.
# stats_df = pd.DataFrame.from_dict(aggregated_data, orient='index')

# # Display the DataFrame
# stats_df.head(20)

### Plot evolution of metrics of previous dataframe

In [ ]:
# def plot_stats_over_years(df, columns):
#     """
#     Plots the evolution of specified columns from a DataFrame over the years.
    
#     Parameters:
#     - df: pandas DataFrame with a year index and columns to plot.
#     - columns: list of strings representing the column names to plot.
#     """
#     if not isinstance(columns, list):
#         columns = [columns]  # Convert to list if a single column name is passed.
        
#     for column in columns:
#         if column in df.columns:
#             plt.plot(df.index, df[column], label=column)
#         else:
#             print(f"Column '{column}' not found in the DataFrame.")
    
#     plt.xlabel('Year')
#     plt.ylabel('Value')
#     plt.title('Evolution of Statistics Over Years')
#     plt.legend()
#     plt.show()

# plot_stats_over_years(stats_df, ['total_like_count', 'total_dislike_count'])
# plot_stats_over_years(stats_df, ['mean_dislike_count', 'mean_like_count', 'mean_duration'])